# Optimalizace kódu

Cílem je vytvořit agenta, kterému dáte funkci v C/C++, návod jak vygenerovat vstup, a agent vám tuto funkci skompiluje, změří její rychlost, a navrhne optimalizované řešení. S agentem lze pracovat iterativně, kdy mu radíte, co ještě vyzkoušet, a agent bude dále kompilovat a spouštět alternativní řešení.

Kód kompilujte jednoduše, ať nemusíte instalovat fancy věci -- s gcc lze použít např.
```
g++ -O3 -o bench bench.cpp -fopenmp
```
Jazykovému modelu je pak vhodné prozradit, co smí používat, či přímo jak kód kompilujete. Pokročilejší verze pak může nechat LLM měnit způsob kompilace.

Agenta testujte na něčem jednoduchém. Aby se jeho doladěnější verze trochu "zabavily", použijte už složitější kód Coulombovské sumace:
```
struct sAtom{
    float x, y, z;
    float w;
};
void coulombNaive(const sAtom* __restrict__ atoms, const int numberOfAtoms, const float gridSpacing, const int gridPoints, float* __restrict__ grid) {
    for (int a = 0; a < numberOfAtoms; a++) {
        sAtom myAtom = atoms[a];
        for (int x = 0; x < gridPoints; x++) {
            float dx2 = powf((float)x * gridSpacing - myAtom.x, 2.0f);
            for (int y = 0; y < gridPoints; y++) {
                float dy2 = powf((float)y * gridSpacing - myAtom.y, 2.0f);
                for (int z = 0; z < gridPoints; z++) {
                    float dz = (float)z * gridSpacing - myAtom.z;
                    float e = myAtom.w / sqrtf(dx2 + dy2 + dz*dz);
                    grid[z*gridPoints*gridPoints + y*gridPoints + x] += e;
                }
            }
        }
    }
```
Do pole atoms můžete pro testování nasypat náhodná čísla, ideálně v intervalu (0, 1), gridSpacing nastavte na 0.01. Dostatečně dlouhý výpočet chce pole grid alokované na velikost 256x256x256, tím pádem gridPoints nastavte na 256, počet atomů numberOfAtoms nastavte okolo 1000 a adekvátně tomu alokujte pole atoms. Pole grid alokujte vynulované.

**Upozornění** v doporučené velikosti vstupu trvá naivní implementace několik minut, nicméně optimalizované verze lze zrychlit na běh v řádu nízkých jednotek sekund na běžném notebooku, proto je kvůli přesnosti měření doporučen relativně velký vstup.

In [ ]:
from openai import OpenAI
import json
import subprocess
import os
from openai import ChatCompletion

In [ ]:
#API key
OPENAI_API_KEY = "TODO"

In [ ]:
client = OpenAI(api_key=OPENAI_API_KEY)